## This jupyter notebook will take a directory full of raw ADNI data and preprocess it in-place.

### In particular, this script: 
### $\bullet$ Uses MRIcron to convert the .dcm filed into .nii.gz files. (Google for installation help)
### $\bullet$ Uses FSL to affine register and skull strip the samples. (See fsl_anat on FSL's website)

In [82]:
import numpy as np
import scipy.io as sio
import os
import pydicom
from subprocess import call
import re

In [83]:
regex = re.compile('(.*gz$)')

In [13]:
patient_condition = 'AD' # set this to 'NC' to traverse the other folder

adni_file_path = "" + os.getcwd() + "/ADNI"
print(adni_file_path)

listFilesDCM = []
folders_level1 = os.listdir(adni_file_path)

numSamples = len(folders_level1)
print(numSamples)

/home/parallels/Alz-Finders/ADNI
43


In [99]:
counter = 0
for sample_idx in range(len(folders_level1)):
    sample = folders_level1[sample_idx]
    sample_file_path = os.path.join(adni_file_path, sample)
    root_sample = sample_file_path
    sample_file_path = os.path.join(sample_file_path, "MPRAGE")
    ref_sample_file_path = sample_file_path
    
    for jj in range(len(os.listdir(sample_file_path))):
        sample_file_path = os.path.join(ref_sample_file_path, os.listdir(ref_sample_file_path)[jj])      
        
        for ii in range(len(os.listdir(sample_file_path))):
            sample_file_path = os.path.join(sample_file_path, os.listdir(sample_file_path)[ii])
            
            #Check if there is already a .nii file. If so, skip this folder.
            breakout = False
            for nii in os.listdir(sample_file_path):
                if regex.match(nii):
                    breakout = True
            if breakout:
                pass
                counter += 1
                break
            
            #Run the conversion tool for this directory of dmc files.
            #call(["dcm2nii", "-v", "y", "-o", root_sample, sample_file_path])
            call(["dcm2nii", "-v", "y", sample_file_path])
            
            print(str(counter) + " " + str(sample_file_path))
            counter += 1
            

    

/home/parallels/Alz-Finders/ADNI/022_S_0129/MPRAGE/2006-02-06_11_54_32.0/S11485
28
/home/parallels/Alz-Finders/ADNI/022_S_0129/MPRAGE/2008-02-19_10_33_04.0/S45881
29
/home/parallels/Alz-Finders/ADNI/036_S_0760/MPRAGE/2007-09-27_11_57_36.0/S40122
30
/home/parallels/Alz-Finders/ADNI/036_S_0760/MPRAGE/2006-08-25_11_11_22.0/S18264
31
/home/parallels/Alz-Finders/ADNI/036_S_0760/MPRAGE/2008-10-07_12_46_02.0/S57028
32
/home/parallels/Alz-Finders/ADNI/023_S_1289/MPRAGE/2007-02-12_11_41_49.0/S26374
33
/home/parallels/Alz-Finders/ADNI/013_S_0996/MPRAGE/2006-11-06_14_53_40.0/S22240
34
/home/parallels/Alz-Finders/ADNI/013_S_0996/MPRAGE/2008-11-17_13_37_17.0/S60124
35
/home/parallels/Alz-Finders/ADNI/023_S_0093/MPRAGE/2008-01-28_15_05_27.0/S45041
36
/home/parallels/Alz-Finders/ADNI/023_S_0093/MPRAGE/2006-01-03_08_54_30.0/S10736
37
/home/parallels/Alz-Finders/ADNI/023_S_0093/MPRAGE/2007-02-13_10_37_18.0/S26504
38
/home/parallels/Alz-Finders/ADNI/036_S_0577/MPRAGE/2006-05-26_12_09_33.0/S14974
39
/hom

In [ ]:
counter = 0
already = 0
unproc = 0

for sample_idx in range(len(folders_level1)):
    sample = folders_level1[sample_idx]
    sample_file_path = os.path.join(adni_file_path, sample)
    root_sample = sample_file_path
    
   
    sample_file_path = os.path.join(sample_file_path, "MPRAGE")
    #print(os.listdir(sample_file_path))
    ref_sample_file_path = sample_file_path
    
    for jj in range(len(os.listdir(sample_file_path))):
        sample_file_path = os.path.join(ref_sample_file_path, os.listdir(ref_sample_file_path)[jj])
        #print("1 " + str(sample_file_path))
        #If this folder has already been preprocessed
        if (os.listdir(ref_sample_file_path)[jj] + ".anat") in os.listdir(ref_sample_file_path):
            #print(str(os.listdir(ref_sample_file_path)[jj]) + "   Already Processed ")
            already +=1
            
            continue
            
        if (sample_file_path[-1]) == "t":
                #print(sample_file_path + "  SKIPPED")
                counter += 1
                continue
        
            
        for ii in range(len(os.listdir(sample_file_path))):
            
            out = (sample_file_path)  
            #print(sample_file_path)
            #print(ii)
            sample_file_path = os.path.join(sample_file_path, os.listdir(sample_file_path)[ii])           
            breakout = False
            #Don't try to preproccess the output folders
            if (sample_file_path[-1]) == "z":
                counter += 1
                continue
                
            #print(sample_file_path)
            
            for nii in os.listdir(sample_file_path):
                if counter <= 2:
                    counter += 1
                    #print("skipped " + str(sample_file_path))
                    break
                if regex.match(nii):
                    breakout = True
                    #print(nii)
                    In = os.path.join(sample_file_path,nii)
                    #print("UNPROCESSED : " + str(unproc) + " " + str(In) + " ")
                    unproc += 1
                    print("Processing " + str(In))
                    call(["fsl_anat", "--noseg", "--nosubcortseg", "--nobias", "--nocrop", "--clobber", "-o", out, "-i", In])
                    #print(":) " + str(counter) + "\n")
                    counter += 1
                    
            if breakout:
                counter += 1
                break         
            
            print("\n\n\n\n\n NO .nii FILE FOUND \n\n\n\n\n")
            #counter += 1
            
            break


Processing /home/parallels/Alz-Finders/ADNI/032_S_1101/MPRAGE/2006-11-21_11_41_21.0/S22783/o20061121_112442MPRAGEs401a1004.nii.gz
Processing /home/parallels/Alz-Finders/ADNI/032_S_1101/MPRAGE/2006-11-21_11_41_21.0/S22783/20061121_112442MPRAGEs401a1004.nii.gz
Processing /home/parallels/Alz-Finders/ADNI/032_S_1101/MPRAGE/2006-11-21_11_41_21.0/S22783/co20061121_112442MPRAGEs401a1004.nii.gz
Processing /home/parallels/Alz-Finders/ADNI/003_S_1059/MPRAGE/2006-11-09_17_28_06.0/S22301/20061109_170013MPRAGEs004a001.nii.gz
Processing /home/parallels/Alz-Finders/ADNI/022_S_0129/MPRAGE/2008-02-19_10_33_04.0/S45881/20080219_101334MPRAGEs002a001.nii.gz
Processing /home/parallels/Alz-Finders/ADNI/036_S_0760/MPRAGE/2008-10-07_12_46_02.0/S57028/20081007_122557MPRAGEs005a001.nii.gz
Processing /home/parallels/Alz-Finders/ADNI/013_S_0996/MPRAGE/2008-11-17_13_37_17.0/S60124/20081117_131635MPRAGEs002a001.nii.gz


In [ ]:
counter = 0

for sample_idx in range(len(folders_level1)):
    sample = folders_level1[sample_idx]
    sample_file_path = os.path.join(adni_file_path, sample)
    root_sample = sample_file_path
    
   
    sample_file_path = os.path.join(sample_file_path, "MPRAGE")
    #print(os.listdir(sample_file_path))
    ref_sample_file_path = sample_file_path
    
    for jj in range(len(os.listdir(sample_file_path))):
        sample_file_path = os.path.join(ref_sample_file_path, os.listdir(ref_sample_file_path)[jj])
        
        #print(sample_file_path)
        
        if sample_file_path[-1] == "t":
            print("Skipping " + str(sample_file_path))
                 
            continue
        
        #If this folder has already been preprocessed
        if (os.listdir(ref_sample_file_path)[jj] + ".anat") in os.listdir(ref_sample_file_path):
            print(str(os.listdir(ref_sample_file_path)[jj])+ " has already been preproc'd - " + str(counter))
            counter +=1     
            continue
            
        for ii in range(len(os.listdir(sample_file_path))):
            
            out = (sample_file_path)    
            sample_file_path = os.path.join(sample_file_path, os.listdir(sample_file_path)[ii])           
            breakout = False
            #Don't try to preproccess the output folders
            if (sample_file_path[-1]) == "z":
                counter += 1
                continue
                
            print(sample_file_path)
            
            for nii in os.listdir(sample_file_path):
                if counter <= 2:
                    counter += 1
                    print("skipped " + str(sample_file_path))
                    break
                if regex.match(nii):
                    breakout = True
                    print(nii)
                    In = os.path.join(sample_file_path,nii)
                    print("Processing " + str(In))
                    call(["fsl_anat", "--noseg", "--nosubcortseg", "--nobias", "--nocrop", "--clobber", "-o", out, "-i", In])
                    print(":) " + str(counter) + "\n")
                    counter += 1
                    
            if breakout:
                counter += 1
                break         
            
            print("\n\n\n\n\n NO .nii FILE FOUND \n\n\n\n\n")
            #counter += 1
            
            break
